In [4]:
from itertools import product

In [78]:
def parse_opcode(opcode):
    """
    Parse an opcode, getting the parameter modes and instruction.
    Opcode should be input as integer.
    """
    code = str(opcode)
    l = len(code)
    inst = int(code[-2:])
    modes = []
    for i in range(l-2):
        modes.append(int(code[l-3-i]))
    return inst, modes

class Amplifier():
    def __init__(self, program):
        self.pos = 0 # position pointer
        self.program = [v for v in program]
        
    def run_program(self,inputs):
        exit_code = 0
        pos = self.pos # where are we in the code
        code = self.program
        in_pos = 0
        while True:
            inst, modes = parse_opcode(code[pos])
            if inst == 99: # terminate
                exit_code = 1
                break
            if inst == 1: # add
                modes = modes+(2-len(modes))*[0]
                if modes[0]==0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1]==0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]

                code[code[pos+3]] = v1+v2 
                pos += 4
            if inst == 2: # multiply
                modes = modes+(2-len(modes))*[0]
                if modes[0]==0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1]==0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]
                code[code[pos+3]] = v1*v2 
                pos += 4
            if inst == 3: # store input
                try:
                    code[code[pos+1]] = inputs[in_pos]
                    in_pos += 1
                    pos += 2
                except IndexError: # must wait for additional input
                    break
            if inst == 4: # output value
                modes = modes+(1-len(modes))*[0]
                if modes[0] == 0:
                    v = code[code[pos+1]]
                else:
                    v = code[pos+1]
                output = v
                pos += 2
            if inst == 5: # jump-if-true
                modes = modes+(2-len(modes))*[0]
                if modes[0] == 0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1] == 0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]
                if v1 > 0:
                    pos = v2
                else:
                    pos += 3
            if inst == 6: # jump-if-false
                modes = modes+(2-len(modes))*[0]
                if modes[0] == 0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1] == 0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]
                if v1 == 0:
                    pos = v2
                else:
                    pos += 3
            if inst == 7:
                modes = modes+(3-len(modes))*[0]
                if modes[0] == 0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1] == 0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]
                if v1 < v2:
                    code[code[pos+3]] = 1
                else:
                    code[code[pos+3]] = 0
                pos += 4
            if inst == 8:
                modes = modes+(3-len(modes))*[0]
                if modes[0] == 0:
                    v1 = code[code[pos+1]]
                else:
                    v1 = code[pos+1]
                if modes[1] == 0:
                    v2 = code[code[pos+2]]
                else:
                    v2 = code[pos+2]
                if v1 == v2:
                    code[code[pos+3]] = 1
                else:
                    code[code[pos+3]] = 0
                pos += 4
        self.pos = pos
        return output, exit_code

In [77]:
# loop through each possible input, ignore ones where they aren't all distinct
# maybe circle back and do this more efficiently
with open("p7_input.txt","r") as f:
    program = [int(s) for s in f.readline().split(",")]
n_amps = 5
max_out_1 = 0
max_out_2 = 0
para1 = [tup for tup in product(*(5*[range(5)])) if len(set(tup))==5]
para2 = [tup for tup in product(*(5*[range(5,10)])) if len(set(tup))==5]

for tup in para1:
    amps = [Amplifier(program) for _ in range(n_amps)]
    output, exit_code = amps[0].run_program([tup[0],0])

    for j in range(n_amps-1):
        output, exit_code = amps[j+1].run_program([tup[j+1],output])
        
    max_out_1 = max(max_out_1,output)

for tup in para2:
    amps = [Amplifier(program) for _ in range(n_amps)]
    output, exit_code = amps[0].run_program([tup[0],0])

    for j in range(n_amps-1):
        output, exit_code = amps[j+1].run_program([tup[j+1],output])
        
    #max_out_1 = max(max_out_1,output)
    
    while exit_code == 0:
        for j in range(n_amps):
            output, exit_code = amps[j].run_program([output])
    max_out_2 = max(max_out_2,output)
    
print("Part 1 solution: {}".format(max_out_1))
print("Part 2 solution: {}".format(max_out_2))

Part 1 solution: 51679
Part 2 solution: 19539216
